In [1]:
import pandas as pd
import geopandas as gpd

gdf_full = gpd.read_feather(r"D:\World Bank\CLIENT v2\Data\Data_proc\WB_country_IDs.feather")

In [3]:
# Generados Nico
gdf = gdf_full.drop(columns=["OBJECTID", "geometry"])
gdf.columns = gdf.columns.str.lower()
gdf = gdf.rename(columns={"id":"ID"})

for shock in ["drought", "floods", "hurricanes"]:
    df = pd.read_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\WB_{shock}_long.csv")

    dflong = df.drop(columns=["Unnamed: 0", "chunk"]).melt(id_vars=["ID", "year", "variable", "threshold"], var_name="measure", value_name="value")
    print(dflong.measure.unique())
    dflong = gdf.merge(dflong, on="ID", validate="1:m").drop(columns=["ID"]) 
    dflong.to_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_{shock}.csv", index=False)

['area_affected' 'population_affected']
['area_affected' 'population_affected']
['area_affected' 'population_affected']


In [18]:
dflong

,adm2_code,adm1_code,adm0_code,year,variable,threshold,measure,value
0,0.0,334,3,1970,category 3,b10,area_affected,0.0
1,0.0,334,3,1970,category 3,b100,area_affected,0.0
2,0.0,334,3,1970,category 3,b25,area_affected,0.0
3,0.0,334,3,1970,category 3,b50,area_affected,0.0
4,0.0,334,3,1970,category 4,b10,area_affected,0.0
...,...,...,...,...,...,...,...,...
46650307,191425.0,2212,182,2020,category 4,b50,population_affected,0.0
46650308,191425.0,2212,182,2020,category 5,b10,population_affected,0.0
46650309,191425.0,2212,182,2020,category 5,b100,population_affected,0.0
46650310,191425.0,2212,182,2020,category 5,b25,population_affected,0.0


In [4]:
# Mapas
gdf_full.drop(columns=["OBJECTID", "ID"]).to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_map.csv", index=False)

In [54]:
## Generados por Yane
# Coldwave
df = pd.read_stata(r"D:\World Bank\CLIENT v2\Data\Data_proc\shocks_CLIENTv1_WB\coldwave_v5_complete.dta")
melt = df.fillna(0).melt(id_vars=["adm0", "adm1", "adm2", "year"])
melt["variable_original"] = melt["variable"]
melt["measure"] = melt["variable_original"].str[2:4].replace({"aa":"area_affected", "ap": "population_affected"})
melt["threshold"] = melt["variable"].str.extract(r"(\d+)").astype(float)
melt["threshold"] = melt["threshold"] / 10
melt["minmax"] = melt["variable"].str[4:7]
melt.loc[melt["minmax"] == "min", "variable"] = "minimum_temperature_below_sd"
melt.loc[melt["minmax"] == "max", "variable"] = "maximum_temperature_below_sd"
melt = melt.drop(columns=["minmax"])
melt.to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_coldwave.csv", index=False)

In [58]:
## Generados por Yane
# Heatwave
df = pd.read_stata(r"D:\World Bank\CLIENT v2\Data\Data_proc\shocks_CLIENTv1_WB\heatwave_v5_complete.dta")
melt = df.fillna(0).melt(id_vars=["adm0", "adm1", "adm2", "year"])
melt["variable_original"] = melt["variable"]
melt["measure"] = melt["variable_original"].str[2:4].replace({"aa":"area_affected", "ap": "population_affected"})
melt["threshold"] = melt["variable_original"].str[6:].astype(float) / 10
melt["variable"] = melt["variable_original"].str[:6].replace({
    "hwap35": "heatwave_35deg",
    "hwaa35": "heatwave_35deg",
    "hwap40": "heatwave_40deg",
    "hwaa40": "heatwave_40deg",
})
melt.drop(columns=["variable_original"]).to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_heatwave.csv", index=False)

In [59]:
## Generados por Yane
# Intenserain
df = pd.read_stata(r"D:\World Bank\CLIENT v2\Data\Data_proc\shocks_CLIENTv1_WB\intenserain_complete.dta")
melt = df.fillna(0).melt(id_vars=["adm0", "adm1", "adm2", "year"])
melt["variable_original"] = melt["variable"]
melt["measure"] = melt["variable_original"].str[1:3].replace({"aa":"area_affected", "ap": "population_affected"})
melt["threshold"] = melt["variable_original"].str[4:].astype(int)
melt["variable"] = melt["variable_original"].str[:4].replace({
    "iap1": "intenserain_1day",
    "iap5": "intenserain_5day",
    "iaa1": "intenserain_1day",
    "iaa5": "intenserain_5day",
})
melt.drop(columns=["variable_original"]).to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_intenserain.csv", index=False)